<h3>Encontrando una estrategia SMA óptima</h3>
Ahora surge la pregunta ¿Podrían existir mejores valores de las medias móviles?, es decir,
¿Habrá un valor diferente a las SMA(50) y SMA(200) que produzca mejores retornos o ganancias?
A continuación se responde esta inquietud.

In [1]:
# se realizan todas las importaciones necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.style.use("seaborn-v0_8-whitegrid")

In [3]:
# se cargan los datos que corresponden al instrumento EUR/USD
df = pd.read_csv("Scripts/eurusd.csv", parse_dates = ["Date"], index_col = ["Date"])

In [4]:
df

,price
Date,
2004-01-01,1.258194
2004-01-02,1.258194
2004-01-05,1.268698
2004-01-06,1.272103
2004-01-07,1.264095
...,...
2020-06-24,1.131235
2020-06-25,1.125087
2020-06-26,1.122208


Ahora vamos a agregar y sumarizar el código creado en la lección anterior para calcular las estrategias B/H (returns) y la otra (strategy). Para esto se va a crear una función que permite
cambiar sobre la marcha los períodos de las medias móviles.

In [5]:
# el argumento de la función es una tupla de dos elementos: el primero es la media móvil corta
# y el segundo es la media móvil larga
def run_strategy(SMA):
    data = df.copy()
    data["returns"] = np.log(data.price.div(data.price.shift(1)))
    data["SMA_S"] = data.price.rolling(int(SMA[0])).mean()
    data["SMA_L"] = data.price.rolling(int(SMA[1])).mean()
    data.dropna(inplace=True)

    data["position"] = np.where(data["SMA_S"] > data["SMA_L"], 1, -1)
    data["strategy"] = data.position.shift(1) * data["returns"]
    data.dropna(inplace=True)

    return data[["returns", "strategy"]].sum().apply(np.exp)#[-1]



In [6]:
# probamos la función con los períodos 50 y 200
run_strategy((50,200))

returns     0.915826
strategy    1.283500
dtype: float64

In [7]:
# ahora con los períodos 10 y 50
run_strategy((10, 50))

returns     0.919492
strategy    0.987000
dtype: float64

Recordar que returns corresponde a B/H (que es el punto de referencia) y strategy es lo que el alemán llama "nuestra estrategia" . Los valores obtenidos con los períodos 50 y 200 son iguales a los obtenidos en la lección anterior.
Cuando se usan los períodos 10 (media móvil corta) y 50 (media móvil larga) se obtienen 0.919492 y 0.987000, o sea dos pérdidas y ambas muy parecidas.

A continuación vamos a probar otra estrategia con los valores 22 y 252. Recordar que 252 es el número de días hábiles en un año:

In [8]:
run_strategy((22, 252))

returns     0.83984
strategy    1.14928
dtype: float64

Los valores anteriores muestran que la estrategia con los períodos 22 y 252 muestran una ganancia pero aún esta ganancia es menor que la obtenida con los períodos 50 y 200.
En este punto es importante resaltar, con respecto a la función que si la instrucción return de la función se escribe en la forma:

         return data[["returns", "strategy"]].sum().apply(np.exp)[-1]

entonces seleccionamos únicamente la realización absoluta de nuestra estrategia (selecciona solamente el último valor de la columna "strategy")

El plan ahora es encontrar una estrategia que maximice la realización absoluta esto lo vamos a hacer usando un algoritmo de optimización de la librería scipy que haga la optimización por fuerza bruta.
El tema ahora es que el optimizador por fuerza bruta de scipy solo puede encontrar el mínimo y no el máximo de la función que es lo que deseamos. Este inconveniente se puede subsanar con un simple truco.
Para ello vamos a redefinir la función run_strategy para que retorne el negativo de la realización absoluta lo cual en la práctica equivale a maximizar la función, que es el resultado deseado.
En la siguiente celda redefinimos la función con los dos cambios:
1. Se calcula el negativo de la realización absoluta
2. Y retorna el último valor de strategy



In [9]:
#redefinición de la función para maximizar el rendimiento
def run_strategy(SMA):
    data = df.copy()
    data["returns"] = np.log(data.price.div(data.price.shift(1)))
    data["SMA_S"] = data.price.rolling(int(SMA[0])).mean()
    data["SMA_L"] = data.price.rolling(int(SMA[1])).mean()
    data.dropna(inplace=True)

    data["position"] = np.where(data["SMA_S"] > data["SMA_L"], 1, -1)
    data["strategy"] = data.position.shift(1) * data["returns"]
    data.dropna(inplace=True)

    return -data[["returns", "strategy"]].sum().apply(np.exp)[-1]

In [10]:
# ahora importamoss la librería para optimizar
from scipy.optimize import brute

In [11]:
brute(run_strategy, ((10, 50, 1), (50, 200, 1)))

C:\Users\marioL\AppData\Local\Temp\ipykernel_11644\768833383.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return -data[["returns", "strategy"]].sum().apply(np.exp)[-1]
C:\Users\marioL\AppData\Local\Temp\ipykernel_11644\768833383.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return -data[["returns", "strategy"]].sum().apply(np.exp)[-1]
C:\Users\marioL\AppData\Local\Temp\ipykernel_11644\768833383.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by po

array([ 46., 137.])

In [12]:
# vamos a ejecutar la función con estos valores
-run_strategy((46,137))

C:\Users\marioL\AppData\Local\Temp\ipykernel_11644\768833383.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return -data[["returns", "strategy"]].sum().apply(np.exp)[-1]


np.float64(2.5266939897810787)

El cálculo anterior se realiza anteponiendo el signo menos al llamado de la función, porque lo que se calcula es la maximización como se explicó antes.
El valor obtenido de 2.53 indica que con esta estrategia se obtienen $2.53 por cada dólar invertido.  
Para analizar este resultado con más detalle se pueden ejecutar los dos notebooks anteriores:  

3.1.2-sma_cross_strategy.ipynb  
3.1.3-vector_strat_back.ipynb  

con los valores óptimos obtenidos de los períodos de las medias móviles corta y larga  (46, 137).  
Haciendo esto se pueden resaltar algunos detalles:  
1. En relación a la columna <code>position</code> se observan algunos cambios cuando comparado con la gráfica obtenida con períodos 50 y 200.. A continuación se muestra la gráfica con períodos 46 y 137 obtenidos de la optimización 

![Gráfica de la posición](position.png)

Usando ahora el notebook de backtesting con los valores optimizados se obtiene un resultado de  
retornos      0.905647  
estrategia    2.526694

y en términos de rendimiento anualizado se obtiene:  
retornos     -0.006035  
estrategia    0.056448  

es decir, un 5.6%, en tanto que el riesgo anualizado es prácticamente el mismo para B/H y la estrategia:  
retornos      0.120418  
estrategia    0.120366  


![comparación de las estrategias](compara.png)

Analizando la gráfica anterior se nota un comportamiento promotedor dado que la realización global luce muy estable durante el tiempo, no como en el caso de medias 50 y 20 en el cual el comportamiento superior de la estrategia frente a B/H, duraba únicamente un par de meses; ahora se puede observar un comportamiento superior durante todo el tiempo considerado.  

En el notebook 3.1.3-vector_strat_back.ipynb también se realiza el cálculo de la superioridad de la estrategia frente a B/H con los valores de los períodos optimizados de las medias móviles el cual arroja un valor de:  
1.6210467811232663  

es decir, 162 puntos porcentuales.

En conclusión esta es la estrategia óptima. No obstante, en este punto debemos tener en cuenta algunas limitaciones del backtesting.

El backtesting es



So whenever a strategy isn't based on past data or isn't optimized this past data, then back testing is appropriate.

But now in this case here, we kind of fitted our strategy to pass data in a way that the strategy is optimized.

And we could say that our algorithm is fitted to the data and there's no guarantee that SMA forty six and one hundred thirty seven is the best strategy in the future as well.

So if a strategy is optimized or fitted to pass data, then we should test the quality of that strategy on future or new, ah, fresh data.

And this is also called further testing.

And we will extensively use our testing in the next sections.

But for the time being, we stop here and try to generalize SMA back testing with an appropriate SMA back test class.

So that's the plan for the next videos.



